## Multiobjective parameter estimation

In [1]:
# Importing the libraries and methods needed to run the notebook.
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sysidentpy.model_structure_selection import FROLS
from sysidentpy.multiobjective.multiobjective_parameter_estimation import IM, AILS
from sysidentpy.basis_function._basis_function import Polynomial
from sysidentpy.utils.display_results import results
from sysidentpy.utils.plotting import plot_results
from sysidentpy.metrics import root_relative_squared_error

data_train = pd.read_csv(r"examples/datasets/buck.txt", sep=" ", header=None).to_numpy()
data_valid = pd.read_csv(r"examples/datasets/buck_val.txt", sep=" ", header=None).to_numpy()
data_samples_train = data_train[:, 2]
data_out_train = data_train[:, 6]
data_int_train = data_train[:, 4]
data_samples_valid = data_valid[:, 2]
data_out_valid = data_valid[:, 6]
data_int_valid = data_valid[:, 4]

Vd = 24
Uo = np.linspace(0, 4, 50)
Yo = (4 - Uo) * Vd / 3
Uo = Uo.reshape(-1, 1)
Yo = Yo.reshape(-1, 1)

gain = -8 * np.ones(len(Uo)).reshape(-1, 1)

# Dynamic Behavior

In [2]:
# Informing test and validation data
x_train = data_int_train.reshape(-1, 1)
y_train = data_out_train.reshape(-1, 1)
x_valid = data_int_valid.reshape(-1, 1)
y_valid = data_out_valid.reshape(-1, 1)

basis_function = Polynomial(degree=2)

model = FROLS(
    order_selection=True,
    n_info_values=8,
    extended_least_squares=False,
    ylag=2,
    xlag=2,
    info_criteria="aic",
    estimator="least_squares",
    basis_function=basis_function,
)

model.fit(X=x_train, y=y_train)

c:\Users\wilso\Desktop\projects\GitHub\gabriel\sysidentpy\sysidentpy\utils\deprecation.py:37: FutureWarning: Passing a string to define the estimator will rise an error in v0.4.0. 
 You'll have to use FROLS(estimator=LeastSquares()) instead. 
 The only change is that you'll have to define the estimator first instead of passing a string like 'least_squares'. 
 This change will make easier to implement new estimators and it'll improve code readability.
  warnings.warn(message, FutureWarning)


In [3]:
mult = IM(model_type=model.model_type, final_model=model.final_model)

In [4]:
# Method to calculate the parameters
J, w, E, Theta, HR, QR, position = mult.affine_information_least_squares(
    y_train=y_train[2:], gain=gain, y_static=Yo, x_static=Uo, psi=model.psi
)
result = {
    "w1": w[0, :],
    "w2": w[2, :],
    "w3": w[1, :],
    "J_ls": J[0, :],
    "J_sg": J[1, :],
    "J_sf": J[2, :],
    "||J||:": E,
}
pd.DataFrame(result)

,w1,w2,w3,J_ls,J_sg,J_sf,||J||:
0,0.990080,0.003078,0.006842,0.992585,9.825994e-02,0.305491,0.261455
1,0.990080,0.002347,0.007573,0.991990,1.551666e-01,0.266590,0.286111
2,0.990080,0.001538,0.008382,0.990885,3.048969e-01,0.220806,0.382836
3,0.990080,0.000642,0.009277,0.988248,1.000000e+00,0.142808,1.000000
4,0.894495,0.098663,0.006842,0.995100,1.365000e-03,0.660484,0.244160
...,...,...,...,...,...,...,...
2290,0.006842,0.333527,0.659632,0.999976,5.934686e-08,0.000020,0.245246
2291,0.006842,0.263039,0.730119,0.999985,6.120704e-08,0.000018,0.245248
2292,0.006842,0.185020,0.808139,0.999993,6.454692e-08,0.000016,0.245250
2293,0.006842,0.098663,0.894495,1.000000,7.415988e-08,0.000015,0.245252


In [5]:
position

29

In [6]:
print(E, position)

[0.26145464 0.28611071 0.38283631 ... 0.24524976 0.24525154 0.24524438] 29


In [7]:
# Writing the results
model.theta = Theta[position, :].reshape(-1, 1)
yhat = model.predict(X=x_valid, y=y_valid)
rrse = root_relative_squared_error(y_valid, yhat)
r = pd.DataFrame(
    results(
        model.final_model,
        model.theta,
        model.err,
        model.n_terms,
        err_precision=3,
        dtype="sci",
    ),
    columns=["Regressors", "Parameters", "ERR"],
)
r

,Regressors,Parameters,ERR
0,y(k-1),2.9687E-01,9.999E-01
1,y(k-2),6.4693E-01,2.042E-05
2,x1(k-1)y(k-1),4.0624E-03,1.108E-06
3,1,1.5405E+00,4.688E-06
4,y(k-1)^2,2.7671E-01,3.922E-07
5,x1(k-1),-4.1302E-01,8.389E-07
6,y(k-2)y(k-1),-5.3474E-01,5.690E-07
7,y(k-2)^2,2.5832E-01,3.827E-06


In [8]:
# Informing test and validation data
x_train = data_int_train.reshape(-1, 1)
y_train = data_out_train.reshape(-1, 1)
x_valid = data_int_valid.reshape(-1, 1)
y_valid = data_out_valid.reshape(-1, 1)

basis_function = Polynomial(degree=2)

model = FROLS(
    order_selection=True,
    n_info_values=8,
    extended_least_squares=False,
    ylag=2,
    xlag=2,
    info_criteria="aic",
    estimator="least_squares",
    basis_function=basis_function,
)

model.fit(X=x_train, y=y_train)

test = AILS(final_model=model.final_model)

# Method to calculate the parameters
J, w, E, Theta, HR, QR, position = test.affine_information_least_squares(
    y_train=y_train[2:], gain=gain, y_static=Yo, x_static=Uo, psi=model.psi
)
result = {
    "w1": w[0, :],
    "w2": w[2, :],
    "w3": w[1, :],
    "J_ls": J[0, :],
    "J_sg": J[1, :],
    "J_sf": J[2, :],
    "||J||:": E,
}
pd.DataFrame(result)

c:\Users\wilso\Desktop\projects\GitHub\gabriel\sysidentpy\sysidentpy\utils\deprecation.py:37: FutureWarning: Passing a string to define the estimator will rise an error in v0.4.0. 
 You'll have to use FROLS(estimator=LeastSquares()) instead. 
 The only change is that you'll have to define the estimator first instead of passing a string like 'least_squares'. 
 This change will make easier to implement new estimators and it'll improve code readability.
  warnings.warn(message, FutureWarning)


,w1,w2,w3,J_ls,J_sg,J_sf,||J||:
0,0.990080,0.003078,0.006842,1.0,1.0,1.0,0.261455
1,0.894495,0.098663,0.006842,1.0,1.0,1.0,0.244160
2,0.808139,0.185020,0.006842,1.0,1.0,1.0,0.244300
3,0.730119,0.263039,0.006842,1.0,1.0,1.0,0.244412
4,0.659632,0.333527,0.006842,1.0,1.0,1.0,0.244489
...,...,...,...,...,...,...,...
2290,0.006842,0.098663,0.894495,1.0,1.0,1.0,0.245252
2291,0.009277,0.000642,0.990080,1.0,1.0,1.0,0.245183
2292,0.008382,0.001538,0.990080,1.0,1.0,1.0,0.245223
2293,0.007573,0.002347,0.990080,1.0,1.0,1.0,0.245236


In [9]:
# Writing the results
model.theta = Theta[position, :].reshape(-1, 1)
yhat = model.predict(X=x_valid, y=y_valid)
rrse = root_relative_squared_error(y_valid, yhat)
r = pd.DataFrame(
    results(
        model.final_model,
        model.theta,
        model.err,
        model.n_terms,
        err_precision=3,
        dtype="sci",
    ),
    columns=["Regressors", "Parameters", "ERR"],
)
r

,Regressors,Parameters,ERR
0,y(k-1),2.9687E-01,9.999E-01
1,y(k-2),6.4693E-01,2.042E-05
2,x1(k-1)y(k-1),4.0624E-03,1.108E-06
3,1,1.5405E+00,4.688E-06
4,y(k-1)^2,2.7671E-01,3.922E-07
5,x1(k-1),-4.1302E-01,8.389E-07
6,y(k-2)y(k-1),-5.3474E-01,5.690E-07
7,y(k-2)^2,2.5832E-01,3.827E-06


In [10]:
position

1229

In [11]:
print(E, position)

[0.26145463 0.24415994 0.24430003 ... 0.24522336 0.24523648 0.24524434] 1229


In [11]:
a = ["static", "gain"]

for i in enumerate(a):
    print(i)

(0, 'static')
(1, 'gain')


In [21]:
np.column_stack([y_train, y_train])

array([[14. , 14. ],
       [14. , 14. ],
       [14.2, 14.2],
       ...,
       [15.4, 15.4],
       [15.4, 15.4],
       [15.6, 15.6]])

In [15]:
gain

array([[-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.],
       [-8.]])